In [35]:
import pandas as pd
from pathlib import Path
from matplotlib import pyplot as plt
import numpy as np

In [39]:
import pandas as pd
from PIL import Image
import xml.etree.ElementTree as ET

def create_DF(files, images_dir):
    """
    Convierte una lista de PosixPaths en un dataframe
    """
    df = pd.DataFrame([])
    for file in files:
        tree = ET.parse(file)
        root = tree.getroot()

        image_dic = []

        for obj in root.findall('object'):
            label = obj.find('name').text       # pineapple
            image = file.stem + ".png"          # fruit0.png
            bndbox = obj.find('bndbox')         
            xmin = float(bndbox.find('xmin').text)      # 38
            ymin = float(bndbox.find('ymin').text)     # 18
            xmax = float(bndbox.find('xmax').text)     # 271)
            ymax = float(bndbox.find('ymax').text)      # 227)

            image_dic.append({
                "label" : label,
                "left" : xmin, 
                "top" : ymin, 
                "right" : xmax,
                "bottom" : ymax
            })
        
        aux_df = pd.DataFrame(image_dic)
        
        # try:
        image_file = next(images_dir.glob(f"**/{file.stem}*.png"))
        aux_df["height"] = root.find('size').find('height').text
        aux_df["width"] = root.find('size').find('width').text
        aux_df["image"] = image_file
        aux_df["image_name"] = root.find('filename').text
        df = pd.concat([df, aux_df])
        # except:
        #     print(file.stem)
    print(df)
    df["center_x"] = (df["right"] + df["left"])/2
    df["center_y"] = (df["bottom"] + df["top"])/2
    df["delta_x"] = df["right"] - df["left"]
    df["delta_y"] = df["bottom"] - df["top"]

    return df

In [69]:
train_files = list(Path("dataset/annotations/").glob("**/*xml"))
train_files2 = list(Path("dataset/annotations/").glob("**/*xml"))
# val_files = list(Path("dataset/annotations/").glob("**/*xml"))

print(train_files)

[WindowsPath('dataset/annotations/fruit0.xml'), WindowsPath('dataset/annotations/fruit1.xml'), WindowsPath('dataset/annotations/fruit10.xml'), WindowsPath('dataset/annotations/fruit100.xml'), WindowsPath('dataset/annotations/fruit101.xml'), WindowsPath('dataset/annotations/fruit102.xml'), WindowsPath('dataset/annotations/fruit103.xml'), WindowsPath('dataset/annotations/fruit104.xml'), WindowsPath('dataset/annotations/fruit105.xml'), WindowsPath('dataset/annotations/fruit106.xml'), WindowsPath('dataset/annotations/fruit107.xml'), WindowsPath('dataset/annotations/fruit108.xml'), WindowsPath('dataset/annotations/fruit109.xml'), WindowsPath('dataset/annotations/fruit11.xml'), WindowsPath('dataset/annotations/fruit110.xml'), WindowsPath('dataset/annotations/fruit111.xml'), WindowsPath('dataset/annotations/fruit112.xml'), WindowsPath('dataset/annotations/fruit113.xml'), WindowsPath('dataset/annotations/fruit114.xml'), WindowsPath('dataset/annotations/fruit115.xml'), WindowsPath('dataset/anno

In [73]:
df_train = create_DF(train_files, Path("dataset/images/"))

           label   left    top  right  bottom height width  \
0      pineapple   38.0   82.0  271.0   227.0    300   400   
1    snake fruit  244.0  174.0  280.0   207.0    300   400   
2   dragon fruit  254.0  228.0  351.0   300.0    300   400   
0      pineapple   38.0   87.0  275.0   241.0    300   400   
1    snake fruit  240.0  185.0  279.0   220.0    300   400   
..           ...    ...    ...    ...     ...    ...   ...   
1    snake fruit  144.0  181.0  162.0   204.0    300   400   
2         banana  204.0  217.0  253.0   262.0    300   400   
0      pineapple  155.0   80.0  261.0   175.0    300   400   
1    snake fruit  143.0  184.0  161.0   208.0    300   400   
2         banana  203.0  222.0  257.0   269.0    300   400   

                         image   image_name  
0    dataset\images\fruit0.png   fruit0.png  
1    dataset\images\fruit0.png   fruit0.png  
2    dataset\images\fruit0.png   fruit0.png  
0    dataset\images\fruit1.png   fruit1.png  
1    dataset\images\fruit

In [1]:
# df_val = create_DF(val_files,Path("Dataset/validation/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [42]:
df_train.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,pineapple,38.0,82.0,271.0,227.0,300,400,dataset\images\fruit0.png,fruit0.png,154.5,154.5,233.0,145.0
1,snake fruit,244.0,174.0,280.0,207.0,300,400,dataset\images\fruit0.png,fruit0.png,262.0,190.5,36.0,33.0
2,dragon fruit,254.0,228.0,351.0,300.0,300,400,dataset\images\fruit0.png,fruit0.png,302.5,264.0,97.0,72.0
0,pineapple,38.0,87.0,275.0,241.0,300,400,dataset\images\fruit1.png,fruit1.png,156.5,164.0,237.0,154.0
1,snake fruit,240.0,185.0,279.0,220.0,300,400,dataset\images\fruit1.png,fruit1.png,259.5,202.5,39.0,35.0


In [52]:
valores_unicos_columna2 = df_train['image_name'].unique()

np.random.shuffle(valores_unicos_columna2)

df_val = df_train[df_train['image_name'].isin(valores_unicos_columna2[:20])]
df_train = df_train[~df_train['image_name'].isin(valores_unicos_columna2[:20])]

In [60]:
df_val.head(100)

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,pineapple,38.0,87.0,275.0,241.0,300,400,dataset\images\fruit1.png,fruit1.png,156.5,164.0,237.0,154.0
1,snake fruit,240.0,185.0,279.0,220.0,300,400,dataset\images\fruit1.png,fruit1.png,259.5,202.5,39.0,35.0
2,dragon fruit,256.0,244.0,346.0,300.0,300,400,dataset\images\fruit1.png,fruit1.png,301.0,272.0,90.0,56.0
0,pineapple,37.0,47.0,225.0,165.0,300,400,dataset\images\fruit100.png,fruit100.png,131.0,106.0,188.0,118.0
1,snake fruit,203.0,176.0,241.0,202.0,300,400,dataset\images\fruit100.png,fruit100.png,222.0,189.0,38.0,26.0
2,banana,285.0,97.0,342.0,156.0,300,400,dataset\images\fruit100.png,fruit100.png,313.5,126.5,57.0,59.0
0,pineapple,92.0,118.0,222.0,213.0,300,400,dataset\images\fruit112.png,fruit112.png,157.0,165.5,130.0,95.0
1,snake fruit,211.0,225.0,238.0,244.0,300,400,dataset\images\fruit112.png,fruit112.png,224.5,234.5,27.0,19.0
2,banana,266.0,145.0,310.0,196.0,300,400,dataset\images\fruit112.png,fruit112.png,288.0,170.5,44.0,51.0
0,pineapple,115.0,76.0,260.0,226.0,300,400,dataset\images\fruit113.png,fruit113.png,187.5,151.0,145.0,150.0


In [64]:
classes = df_train["label"].unique()
classes

array(['pineapple', 'snake fruit', 'dragon fruit', 'banana'], dtype=object)

In [65]:
classes_idx = {clase:idx for idx,clase in enumerate(classes)}

In [66]:
cant_classes = len(classes)
cant_classes

4

In [67]:
train_files = list(Path("dataset/annotations/").glob("**/*xml"))
# val_files = list(Path("dataset/annotations/").glob("**/*xml"))

print(train_files)

[WindowsPath('dataset/annotations/fruit0.xml'), WindowsPath('dataset/annotations/fruit1.xml'), WindowsPath('dataset/annotations/fruit10.xml'), WindowsPath('dataset/annotations/fruit100.xml'), WindowsPath('dataset/annotations/fruit101.xml'), WindowsPath('dataset/annotations/fruit102.xml'), WindowsPath('dataset/annotations/fruit103.xml'), WindowsPath('dataset/annotations/fruit104.xml'), WindowsPath('dataset/annotations/fruit105.xml'), WindowsPath('dataset/annotations/fruit106.xml'), WindowsPath('dataset/annotations/fruit107.xml'), WindowsPath('dataset/annotations/fruit108.xml'), WindowsPath('dataset/annotations/fruit109.xml'), WindowsPath('dataset/annotations/fruit11.xml'), WindowsPath('dataset/annotations/fruit110.xml'), WindowsPath('dataset/annotations/fruit111.xml'), WindowsPath('dataset/annotations/fruit112.xml'), WindowsPath('dataset/annotations/fruit113.xml'), WindowsPath('dataset/annotations/fruit114.xml'), WindowsPath('dataset/annotations/fruit115.xml'), WindowsPath('dataset/anno

In [4]:
# df_val = create_DF(val_files,Path("Dataset/validation/Apple_Grape_Cantaloupe_Watermelon_Pomegranate_Pineapple_Peach_Mango_Banana_Lemon_Tomato_Strawberry_Pear/"))

In [6]:
df_val.head()

,label,left,top,right,bottom,height,width,image,image_name,center_x,center_y,delta_x,delta_y
0,Tomato,72.387584,119.736162,793.486336,619.030503,1024,819,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,604835415b761f30,432.936960,369.383333,721.098752,499.294341
1,Lemon,89.567232,152.054784,441.594880,461.195520,1024,768,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,b39532c4a3bd5d83,265.581056,306.625152,352.027648,309.140736
2,Strawberry,0.000000,0.053248,1023.358976,1024.000000,1024,1024,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,e0be14f0681f4b39,511.679488,512.026624,1023.358976,1023.946752
3,Lemon,236.441600,90.215200,905.010176,573.973625,1024,575,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,730020d358ea3d82,570.725888,332.094412,668.568576,483.758425
4,Lemon,320.348160,30.697525,724.350976,119.152075,1024,575,Dataset/validation/Apple_Grape_Cantaloupe_Wate...,730020d358ea3d82,522.349568,74.924800,404.002816,88.454550


In [11]:
classes_idx = {clase:idx for idx,clase in enumerate(classes)}

In [71]:
import pickle
with open("data.pkl","wb") as f:
    pickle.dump((df_train, df_val, train_files, classes_idx, cant_classes, classes),f)